# Running pre-trained models on biomedical texts to evaluate them
## Infrastructure

In [ ]:
from google.colab import drive

drive.mount('/content/drive')
%cd /content/drive/MyDrive/UPM/Thesis/Experiments/Train

In [ ]:
!pip install transformers tqdm sentencepiece sacremoses accelerate ipywidgets protobuf
# pytorch

In [ ]:
from abc import abstractmethod


class TranslationModel:
    def __init__(self, checkpoint_name: str):
        self.checkpoint_name = checkpoint_name

    @abstractmethod
    def translate(self, source: str) -> str:
        """Translates a source text with the model
        :param source: the text to translate
        :return: str - the translation"""
        pass

    def __str__(self):
        return self.checkpoint_name

### Helsinki-NLP/opus-mt-en-es

In [ ]:
from transformers import MarianMTModel, MarianTokenizer


class HelsinkiNLPModel(TranslationModel):
    def __init__(self):
        super().__init__("Helsinki-NLP/opus-mt-en-es")
        self.model = MarianMTModel.from_pretrained(self.checkpoint_name)
        self.tokenizer = MarianTokenizer.from_pretrained(self.checkpoint_name)

    def translate(self, source: str) -> str:
        input_ids = self.tokenizer.encode(source, return_tensors="pt")
        translated_tokens = self.model.generate(input_ids, num_beams=4, early_stopping=True)
        translated_text = self.tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
        return translated_text

## Loading and evaluation functions

In [ ]:
import json

import pandas as pd
from tqdm import tqdm


def load_sentences(test_dataset: str, num_rows=500, seed=42) -> pd.DataFrame:
    """Loads a test dataset in .jsonl format into a dataframe and randomly selects n rows
    :param test_dataset: filename of the test dataset
    :param num_rows: number of rows to select
    :param seed: random seed for reproducibility
    :return pd.Dataframe: the parallel corpus as a dataframe"""
    data = []
    with open(test_dataset, "r") as f:
        for line in f:
            loaded = json.loads(line)
            english, spanish = loaded['en'].strip('"'), loaded['es'].strip('"')
            data.append({"en": english, "es": spanish})
    df = pd.DataFrame(data)
    return df.sample(n=num_rows, random_state=seed)


def run_model(model: TranslationModel, test_sentences: pd.DataFrame):
    """Runs a model on the test sentences. Creates a dataframe for the results with two columns: 'Reference' and 'Actual'
    :param model: the MT model to evaluate
    :param test_sentences: a dataframe for a parallel corpus"""
    df = pd.DataFrame(columns=['reference', 'actual'])

    for _, row in tqdm(test_sentences.iterrows()):
        english, spanish = row['en'], row['es']
        translation = model.translate(english)
        df.loc[len(df.index)] = [spanish, translation]

    return df


def evaluate_on_all_test_data(translation_model: TranslationModel, dataset_names: list[str],
                              corpus_directory: str):
    for test_dataset in dataset_names:
        test_path = corpus_directory + test_dataset
        print(f"Test dataset: {test_dataset}")
        test_sentences = load_sentences(test_path, num_rows=500)
        results = run_model(translation_model, test_sentences)
        results.to_csv("helsinki-nlp/" + test_dataset.split(".jsonl")[0] + ".csv", index=False)

## Evaluating the models

In [ ]:
test_directory = "data/"
filenames = ["pubmed-tr.jsonl", "clinspen-tr.jsonl", "khresmoi-tr.jsonl", "medline.jsonl", "orphanet-definitions-tr.jsonl", "orphanet-terms.jsonl", "preferred-en2es.jsonl", "snomed.jsonl"]

In [ ]:
evaluate_on_all_test_data(HelsinkiNLPModel(), filenames, test_directory)